# Hypotheses Ontology

In [1]:
## Uncomment if you do not have owlrl installed (you should have it installed from the RDFS tutorial)
import sys
# !{sys.executable} -m pip install rdflib  owlready2 pandas

import pandas as pd
from rdflib.namespace import DC, FOAF, NamespaceManager

from owlready2 import *
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL, RDFS

# documentation : https://pythonhosted.org/Owlready2/

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Loading and creating ontologies and namespaces.

In [2]:
# TODO: Change that
hypothesis_uri = URIRef("http://test.org/myonto.owl#")
hypothesis_namespace = Namespace(hypothesis_uri)
hypothesis_ontology = get_ontology(hypothesis_uri)

dct_uri = URIRef("http://purl.org/dc/terms/")
dct_namespace = Namespace(dct_uri)
# File from webpage did not work for me -- I downloaded it.
dct_ontology = get_ontology("dublin_core_terms.rdf").load()

frbr_uri = URIRef("http://purl.org/vocab/frbr/core#")
frbr_namespace = Namespace(frbr_uri)
# File from webpage did not wrk for me -- I downloaded it.
frbr_ontology = get_ontology("./frbr-core-20050810.rdf").load()

owl_uri = URIRef("http://www.w3.org/2002/07/owl#")
owl_namespace = Namespace(owl_uri)

fabio_uri = URIRef("http://purl.org/spar/fabio/")
fabio_namespace = Namespace(fabio_uri)

prov_uri = URIRef("http://www.w3.org/ns/prov#")
prov_namespace = Namespace(prov_uri)
# File from webpage did not work for me -- I downloaded it.
prov_ontology = get_ontology("prov_ontology.rdf").load()


g = Graph()
g.parse("cord19-nekg-dataset-metadata.ttl", format="ttl")

cord19_nekg_dataset_metadata_file = open("cord19-nekg-dataset-metadata.rdf", "w+")
cord19_nekg_dataset_metadata_file.write(g.serialize(format='xml').decode())
cord19_nekg_dataset_metadata_file.close()

covid_uri = URIRef("http://ns.inria.fr/covid19/")
covid_namespace = Namespace(covid_uri)
covid_ontology = get_ontology("cord19-nekg-dataset-metadata.rdf").load()

covidpr_uri = URIRef("http://ns.inria.fr/covid19/property")
covidpr_namespace = Namespace(covidpr_uri)

* Owlready2 * WARNING: ObjectProperty http://www.w3.org/ns/prov#wasRevisionOf belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, prov_ontology.wasDerivedFrom]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://www.w3.org/ns/prov#specializationOf belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, prov_ontology.alternateOf, owl.topObjectProperty]; I'm trying to fix it...


# Ontology classes and properties

## 1. Classes

In [9]:
hypothesis_ontology.label = "ADASDASSDA"

with hypothesis_ontology:
    owlready2.comment = "ASDAS"
    comment = "ADSAD"
    class Hypothesis(Thing):
        label = "Hypothesis"
        comment = "Paper's hypothesis."
    #   Will include dct:Title, bibo:doi, bibo:pmid, fabio:hasPubMedId, fabio:hasPubMedCentralId, schema:url, foaf:sha1
    #   Will link with fabio:Abstract by oa:hasSource (TODO: change in graph)

    # ConfidenceScore will be linked by owl:dataProperty which links with simple datatypes, not entities, so class is not needed
    # class ConfidenceScore(Thing):
    #     namespace = hypothesis_ontology

    class NamedEntity(Thing, prov_ontology.Entity):
        label = "Named Entity"
        comment = "Named entity included in the paper's abstract."
    #   Will contain dct:Subject, rdf:value, oa:hasBody
    #   TODO: how do we name these NameEntity-ies?

    class NamedEntitySet(Thing):
        label = "Named Entity set"
        comment = "Set of entities included in the paper's abstract."

## 2. Properties

In [10]:
with hypothesis_ontology:
    class containsEntity(ObjectProperty, FunctionalProperty, Hypothesis >> NamedEntitySet):
        label = "Resource representing a hypothesis's entity set."
        comment = "Links hypothesis with its entity set."
        
    class hasHypothesis(ObjectProperty, list(frbr_ontology.properties())[45], FunctionalProperty):
        label = "Resource representing a document's hypothesis entity."
        comment = "Links document with its hypothesis entity."
        range = [Hypothesis,]

    class includes(ObjectProperty, NamedEntitySet >> NamedEntity):
        label = "Resource representing a Named Entity in the set."
        comment = "Links Named Entity set with its elements."

## 2.5. Restrictions

In [11]:
with hypothesis_ontology:
    NamedEntitySet.equivalent_to = [includes.min(1, NamedEntity), ]
    Hypothesis.equivalent_to = [containsEntity.exactly(1, NamedEntitySet),]

## 3. Graph building

In [12]:
hypothesis_ontology.save(file = "hypothesis_ontology.nt", format = "ntriples") # also supported ntriples
g = Graph()
g.bind("frbr", frbr_namespace)
g.bind("owl", owl_namespace)
g.bind("hypothesis", hypothesis_namespace)
g.bind("fabio", fabio_namespace)
g.parse("hypothesis_ontology.nt", format="nt")

<Graph identifier=Nd05faef713c54b58996fccd34df46d91 (<class 'rdflib.graph.Graph'>)>

## 4. Additional things added to the graph.

In [13]:
# g.add((hypothesis_namespace.hasHypothesis, RDFS.subPropertyOf, frbr_namespace.part))
# g.add((hypothesis_namespace.Abstract, RDFS.subClassOf, fabio_namespace.Work))


## 5. Saving the graph

In [14]:
result = g.serialize(format='turtle').decode()
print(result)

@prefix frbr: <http://purl.org/vocab/frbr/core#> .
@prefix hypothesis: <http://test.org/myonto.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://test.org/myonto.owl> a owl:Ontology .

hypothesis:hasHypothesis a owl:FunctionalProperty,
        owl:ObjectProperty ;
    rdfs:label "Resource representing a document's hypothesis entity."^^xsd:string ;
    rdfs:comment "Links document with its hypothesis entity."^^xsd:string ;
    rdfs:range hypothesis:Hypothesis ;
    rdfs:subPropertyOf frbr:part .

hypothesis:containsEntity a owl:FunctionalProperty,
        owl:ObjectProperty ;
    rdfs:label "Resource representing a hypothesis's entity set."^^xsd:string ;
    rdfs:comment "Links hypothesis with its entity set."^^xsd:string ;
    rdfs:domain hypothesis:Hypothesis ;
    rdfs:range hypothesis:NamedEntitySet .

hypothesis:includes a owl:ObjectProperty ;
    rdfs:label "Res

In [ ]:
hypothesis_owl_file = open("hypothesis_owl.ttl", "w+")
hypothesis_owl_file.write(result)
hypothesis_owl_file.close()